In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium

In [2]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [3]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20
0,Anhui,Mainland China,31.82571,117.2264,1,9,15,39,60,70,...,973,982,986,987,988,989,989,989,989,989
1,Beijing,Mainland China,40.18238,116.4142,14,22,36,41,68,80,...,381,387,393,395,396,399,399,399,400,400
2,Chongqing,Mainland China,30.05718,107.8740,6,9,27,57,75,110,...,553,555,560,567,572,573,575,576,576,576
3,Fujian,Mainland China,26.07783,117.9895,1,5,10,18,35,59,...,290,292,293,293,293,293,293,293,294,294
4,Gansu,Mainland China,36.06110,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,91


In [4]:
#Missing values in each column
print('Confirmed: ', confirmed.isna().sum()[confirmed.isna().sum()>0])
print('recovered: ', recovered.isna().sum()[confirmed.isna().sum()>0])
print('deaths: ', deaths.isna().sum()[confirmed.isna().sum()>0])

Confirmed:  Province/State    41
dtype: int64
recovered:  Province/State    41
dtype: int64
deaths:  Province/State    41
dtype: int64


In [5]:
# replacing missings with Unknown
confirmed = confirmed.fillna('unknow')
recovered = recovered.fillna('unknow')
deaths = deaths.fillna('unknow')

In [6]:
#Last date
last_update = confirmed.columns[-1]
last_update

'2/26/20'

In [7]:
# Varsirus propagation on China
china_cases = confirmed[['Province/State','Lat','Long',last_update]][confirmed['Country/Region']=='Mainland China']
china_cases['recovered'] = recovered[last_update][recovered['Country/Region']=='Mainland China']
china_cases['deaths'] = deaths[last_update][deaths['Country/Region']=='Mainland China']
# renaming columns
china_cases = china_cases.rename(columns = {last_update:'confirmed','recovered':'recovered','deaths':'deaths'})

In [8]:
china_cases.head()

,Province/State,Lat,Long,confirmed,recovered,deaths
0,Anhui,31.82571,117.2264,989,744,6
1,Beijing,40.18238,116.4142,400,235,4
2,Chongqing,30.05718,107.8740,576,384,6
3,Fujian,26.07783,117.9895,294,218,1
4,Gansu,36.06110,103.8343,91,81,2


In [9]:
_ = china_cases.groupby(['Province/State'])['confirmed', 'recovered', 'deaths'].max()
_.style.background_gradient(cmap='PuBu')

,confirmed,recovered,deaths
Province/State,,,
Anhui,989,744,6
Beijing,400,235,4
Chongqing,576,384,6
Fujian,294,218,1
Gansu,91,81,2
Guangdong,1347,851,7
Guangxi,252,147,2
Guizhou,146,104,2
Hainan,168,129,5


In [10]:
world_cases = confirmed[['Country/Region','Province/State',last_update]][confirmed['Country/Region']!='Mainland China']
world_cases['recovered'] = recovered[last_update][recovered['Country/Region']!='Mainland China']
world_cases['deaths'] = deaths[last_update][deaths['Country/Region']!='Mainland China'] 
world_cases = world_cases.rename(columns = {last_update:'Confirmed','recovered':'Recovered','deaths':'Deaths'})
_ = world_cases.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Recovered', 'Deaths'].max()
_.style.background_gradient(cmap='PuBu')

In [11]:
_map = confirmed[['Country/Region','Province/State', 'Lat','Long',last_update]]
_map['Recovered'] = recovered[last_update]
_map['Deaths'] = deaths[last_update]
_map=_map.rename(columns = {last_update:'Confirmed'})
_map = _map.rename(columns = {last_update:'Confirmed','recovered':'Recovered','deaths':'Deaths'})

D:\Anaconda3\envs\backtest_algo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\envs\backtest_algo\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
f_m = folium.Map(location=[30.7378, 112.2384],
               tiles = "CartoDB dark_matter",
               detect_retina = True, zoom_start=6)

for i in range(0, len(china_cases)):
   
    folium.Circle(
        location=[china_cases.iloc[i]['Lat'], china_cases.iloc[i]['Long']],
        color='#81D8D0', 
        tooltip =   
                    '<li><bold>Province : '+str(china_cases.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(china_cases.iloc[i]['confirmed'])+
                    '<li><bold>Recovered : '+str(china_cases.iloc[i]['recovered'])+
                    '<li><bold>Deaths : '+str(china_cases.iloc[i]['deaths']),
        radius=int( np.sqrt(china_cases.iloc[i]['confirmed']))).add_to(f_m)
f_m

In [12]:
f_m = folium.Map(location=[10, 50],
                 tiles = 'Stamen Toner',
                 detect_retina = True, zoom_start=3)

for i in range(0, len(_map)):
   
    folium.CircleMarker(
        location=[_map.iloc[i]['Lat'], _map.iloc[i]['Long']],
        color='red', 
        popup =    '<bold>Country  '+'<font color="#a9b171">'+str(_map.iloc[i]['Country/Region']).upper()+'</font>'+'<br>'+
                    '<br>'+'<bold>Province  '+'<font color="#a9b171">'+str(_map.iloc[i]['Province/State']).upper()+'</font>'+'<br>'+
                    '<br>'+'<bold>Confirmed  '+'<font color="#c745eb">'+str(_map.iloc[i]['Confirmed'])+'</font>'+'<br>'+
                    '<br>'+'<bold>Recovered  '+'<font color="green">'+str(_map.iloc[i]['Recovered'])+'</font>'+'<br>'+
                    '<br>'+'<bold>Deaths'+'<br>'+'<font color="red">'+str(_map.iloc[i]['Deaths'])+'</font>',
        radius=13,
        fill_color='red',
        fill_opacity=0.7).add_to(f_m)
f_m